# University of Michigan
## School of Information
**Masters of Applied Data Science**<br>
**Milestone 1**<br>
<br>
Affordable Housing Project
<br><br>
**Submitted by**<br>
>Alan Fehsenfeld<br>Koon Leong Ho<br>George Thio

# Add Walkscore Score to Municipality File

In [2]:
import pandas as pd
import numpy as np
import geopandas
from collections import Counter
import requests
import time
from config input basedir
base = basedir()

## Import Block Walkscore File and Code by Looking up API Values

In [3]:
data_dir = base + r"/data"
fp_input_file = data_dir + r"/block_walkscores.csv"

# block_geocodes=pd.read_csv('documents/UofM/SIADS 591/block_walkscores.csv')

block_geocodes=pd.read_csv(fp_input_file)
block_geocodes_for_processing=block_geocodes[block_geocodes.walkscore_ind==0]
block_geocodes_for_processing

,Unnamed: 0,GEOID_BLOCK,INTPTLAT_BLOCK,INTPTLON_BLOCK,walkscore,walkscore_ind


In [ ]:
walkscore_list=[]

for x in block_geocodes_for_processing.index:
    latitude=str(round(block_geocodes_for_processing.loc[x,'INTPTLAT_BLOCK'],4))
    longitude=str(round(block_geocodes_for_processing.loc[x,'INTPTLON_BLOCK'],4))
    response=requests.get('https://api.walkscore.com/score?format=json&lat='+latitude+'&lon='+longitude+'&transit=1&bike=1&wsapikey=c6b0ec0b6dd9e4253db963d7a8ad1365')
    try:
        walkscore_list.append(response.json()['walkscore'])
        print(str(x)+' - ',str(response.json()['walkscore']))
    except KeyError:
        walkscore_list.append(-1)
        print(str(x)+' - '+str(-1))
    time.sleep(1)
    
    
response

## Add Lookup Values to Walkscore Table and Export Walkscore Table


In [ ]:
previously_processed_records=block_geocodes[block_geocodes.walkscore_ind==1]
previously_processed_walkscores=list(previously_processed_records.walkscore.values)
walkscore_list_for_update=previously_processed_walkscores+walkscore_list+list(np.zeros(block_geocodes_for_processing.shape[0]-len(walkscore_list)))
block_geocodes['walkscore']=walkscore_list_for_update

In [ ]:
items_processed_count=len(walkscore_list)+previously_processed_records.shape[0]
walkscore_ind=list(np.ones(items_processed_count))+list(np.zeros(block_geocodes.shape[0]-items_processed_count))
block_geocodes['walkscore_ind']=walkscore_ind
block_geocodes.columns

In [184]:
block_geocodes=block_geocodes.loc[:,['GEOID_BLOCK','INTPTLAT_BLOCK','INTPTLON_BLOCK','walkscore','walkscore_ind']]

data_dir = base + r"/data"
fp_output_file = data_dir + r"/block_walkscores.csv"
block_geocodes.to_csv(fp_output_file)

# block_geocodes.to_csv('documents/UofM/SIADS 591/block_walkscores.csv')

## Read in Walkscore Table and Add City/Township Information

In [187]:
data_dir = base + r"/data"
fp_input_file = data_dir + r"/block_walkscores.csv"

# block_walkscores=geopandas.read_file('documents/UofM/SIADS 591/block_walkscores.csv')

block_walkscores=geopandas.read_file(fp_input_file)
block_walkscores_df=geopandas.GeoDataFrame(block_walkscores,geometry=geopandas.points_from_xy(\
                    block_walkscores.loc[:,'INTPTLON_BLOCK'],block_walkscores.loc[:,'INTPTLAT_BLOCK']))
block_walkscores_df=block_walkscores_df.set_crs('WGS84')
block_walkscores_df=block_walkscores_df.to_crs('EPSG:2253')

/Users/alanfehsenfeld/opt/anaconda3/lib/python3.8/site-packages/geopandas/geodataframe.py:422: RuntimeWarning: Sequential read of iterator was interrupted. Resetting iterator. This can negatively impact the performance.
  for feature in features_lst:


,field_1,GEOID_BLOCK,INTPTLAT_BLOCK,INTPTLON_BLOCK,walkscore,walkscore_ind,geometry
0,0,260050000000000.0,42.764015,-85.59509150000002,0,1.0,POINT (-85.59509 42.76402)
1,1,260150000000000.0,42.769112899999996,-85.38274109999998,0,1.0,POINT (-85.38274 42.76911)
2,2,260670000000000.0,43.0428836,-85.30689170000002,0,1.0,POINT (-85.30689 43.04288)
3,3,260670000000000.0,43.07529870000001,-85.2579615,0,1.0,POINT (-85.25796 43.07530)
4,4,260670000000000.0,42.80607910000001,-85.3008553,0,1.0,POINT (-85.30086 42.80608)
...,...,...,...,...,...,...,...
9142,9142,261180000000000.0,43.3149723,-85.33944749999998,0,1.0,POINT (-85.33945 43.31497)
9143,9143,261180000000000.0,43.2275902,-85.2613436,0,1.0,POINT (-85.26134 43.22759)
9144,9144,261180000000000.0,43.1831943,-85.26993409999999,35,1.0,POINT (-85.26993 43.18319)
9145,9145,261180000000000.0,43.1515355,-85.2727783,1,1.0,POINT (-85.27278 43.15154)


In [192]:
data_dir = base + r"/data/municipalities"
fp_input_file = data_dir + r"/City_and_Township.shp"

# cities_and_townships=geopandas.read_file("documents/UofM/SIADS 591/City_and_Township.shp")

cities_and_townships=geopandas.read_file(fp_input_file)
cities_and_townships=cities_and_townships.to_crs('EPSG:2253')
block_walkscores_with_cities_and_townships=geopandas.sjoin(cities_and_townships,block_walkscores_df,how='inner',op='intersects')

In [193]:
block_walkscores_with_cities_and_townships=block_walkscores_with_cities_and_townships.drop(columns='geometry')

data_dir = base + r"/data"
fp_output_file = data_dir + r"/block_walkscores_with_cities_and_townships.csv"
block_walkscores_with_cities_and_townships.to_csv(fp_output_file)

# block_walkscores_with_cities_and_townships.to_csv('documents/UofM/SIADS 591/block_walkscores_with_cities_and_townships.csv')

In [326]:
block_walkscores_with_cities_and_townships=block_walkscores_with_cities_and_townships.loc[:,['CVTNAME','GEOID_BLOCK','INTPTLAT_BLOCK','INTPTLON_BLOCK','walkscore']]

## Code Municipality Name to Match Municipality from All_Municipalities Table

In [325]:
data_dir = base + r"/data/municipalities"
fp_input_file = data_dir + r"/municipalities file.csv"

# municipalities=pd.read_csv('documents/UofM/SIADS 591/municipalities file.csv')

municipalities=pd.read_csv(fp_input_file)
block_walkscores_with_cities_and_townships2=block_walkscores_with_cities_and_townships.merge(municipalities,\
    how='inner',left_on='CVTNAME',right_on='SHAPE FILE MUNICIPALITY')
block_walkscores_with_cities_and_townships2['walkscore']=\
    block_walkscores_with_cities_and_townships2['walkscore'].apply(int)
block_walkscores_with_cities_and_townships2['MUNICIPALITIES FILE MUNICIPALITY']=\
    block_walkscores_with_cities_and_townships2['MUNICIPALITIES FILE MUNICIPALITY'].astype(str)

## Calculate Maximum Walkscore by Municpality

In [547]:
max_walkscore=block_walkscores_with_cities_and_townships2.loc[:,['MUNICIPALITIES FILE MUNICIPALITY','walkscore']]
max_walkscore=max_walkscore.groupby(['MUNICIPALITIES FILE MUNICIPALITY']).agg('max')
max_walkscore=max_walkscore.rename(columns={'walkscore':'max_walkscore'})
max_walkscore=block_walkscores_with_cities_and_townships2.merge(max_walkscore,how='inner',\
    left_on=['MUNICIPALITIES FILE MUNICIPALITY','walkscore'],right_on=['MUNICIPALITIES FILE MUNICIPALITY',\
    'max_walkscore'])
max_walkscore=max_walkscore.loc[:,['MUNICIPALITIES FILE MUNICIPALITY','INTPTLAT_BLOCK','INTPTLON_BLOCK',\
    'max_walkscore']]

## Find Municipalities and Blocks with Walkscores Greater than 80

In [650]:
max_walkscore_gt80=max_walkscore[max_walkscore['max_walkscore']>=80].groupby('MUNICIPALITIES FILE MUNICIPALITY').agg('count').reset_index()
blocks_gt80=max_walkscore_gt80.merge(block_walkscores_with_cities_and_townships2,how='left',\
                                     left_on='MUNICIPALITIES FILE MUNICIPALITY',\
                                     right_on='MUNICIPALITIES FILE MUNICIPALITY')
blocks_gt80=blocks_gt80[blocks_gt80['walkscore']>=80]
blocks_gt80=geopandas.GeoDataFrame(blocks_gt80,geometry=geopandas.points_from_xy(blocks_gt80.loc[:,'INTPTLON_BLOCK_y'],\
                                                                                blocks_gt80.loc[:,'INTPTLAT_BLOCK_y']))
blocks_gt80=blocks_gt80.set_crs('WGS84')
blocks_gt80=blocks_gt80.to_crs('EPSG:2253')
blocks_gt80_geo=blocks_gt80.buffer(1760)
blocks_gt80.geometry=blocks_gt80_geo

## Import Municipalities File and Join it with  Blocks_gt80

In [292]:
data_dir = base + r"/data/municipalities"
fp_input_file = data_dir + r"/All Municipalities Geometry and Municipality.shp"

Municipalities_file=geopandas.read_file(fp_input_file)

# Municipalities_file=geopandas.read_file('documents/UofM/SIADS 591/All Municipalities Geometry and Municipality.shp')

In [293]:
Municipalities_file_gt80=max_walkscore_gt80.merge(Municipalities_file,how='left',\
                                             left_on='MUNICIPALITIES FILE MUNICIPALITY',right_on='Municipali')
Municipalities_file_gt80=geopandas.GeoDataFrame(Municipalities_file_gt80)
Municipalities_file_gt80=Municipalities_file_gt80.loc[:,['ID','Municipali','geometry']]

In [294]:
Municipalities_file_gt80=geopandas.sjoin(Municipalities_file_gt80,blocks_gt80,how='left',op='intersects')
Municipalities_file_gt80=Municipalities_file_gt80.loc[:,['ID','index_right']]

In [295]:
Municipalities_file_gt80.loc[:,'index_right']=Municipalities_file_gt80.loc[:,'index_right']\
    .where(Municipalities_file_gt80.loc[:,'index_right'].isna(),12)
Municipalities_file_gt80.loc[:,'index_right']=Municipalities_file_gt80.loc[:,'index_right']\
    .where(Municipalities_file_gt80.loc[:,'index_right'].notna(),0)
Municipalities_file_gt80=Municipalities_file_gt80.rename(columns={'index_right':'walkscore_ind'})

In [297]:
gt80=Municipalities_file_gt80.groupby('ID').agg('max').reset_index()

## Find Municipalities with Max Walkscores between 50 and 80

In [558]:
max_walkscore_gt50=max_walkscore[(max_walkscore['max_walkscore']>=50)&(max_walkscore['max_walkscore']<80)]
max_walkscore_cities=max_walkscore_gt50.loc[:,'MUNICIPALITIES FILE MUNICIPALITY'].unique()

max_walkscore_gt50=geopandas.GeoDataFrame(max_walkscore_gt50,geometry=\
                                     geopandas.points_from_xy(max_walkscore_gt50.loc[:,'INTPTLON_BLOCK'],\
                                                              max_walkscore_gt50.loc[:,'INTPTLAT_BLOCK']))
max_walkscore_gt50=max_walkscore_gt50.set_crs('WGS84')
max_walkscore_gt50=max_walkscore_gt50.to_crs('EPSG:2253')

## Find Parcels in Municipalities with Max Walkscores between 50 and 80

In [559]:
data_dir = base + r"/data/municipalities"
fp_input_file = data_dir + r"/All Municipalities Geometry and Municipality.shp"

Municipalities_file_50to80=geopandas.read_file(fp_input_file)

# Municipalities_file_50to80=geopandas.read_file('documents/UofM/SIADS 591/All Municipalities Geometry and Municipality.shp')

In [560]:
Municipalities_file_50to80=Municipalities_file_50to80.merge(max_walkscore_gt50,how='right',left_on='Municipali',right_on='MUNICIPALITIES FILE MUNICIPALITY')

In [561]:
Municipalities_file_50to80=Municipalities_file_50to80.drop_duplicates(subset='ID',keep='first')
Municipalities_file_50to80=Municipalities_file_50to80.loc[:,['ID','Municipali','geometry_x']]
Municipalities_file_50to80=Municipalities_file_50to80.rename(columns={'geometry_x':'geometry'})
geometry=Municipalities_file_50to80.loc[:,'geometry']
Municipalities_file_50to80=geopandas.GeoDataFrame(Municipalities_file_50to80,geometry=geometry)
Municipalities_file_50to80=Municipalities_file_50to80.set_crs('EPSG:2253')

## Find Parcels 1/3rd of a Mile from Max Walkscore

In [562]:
max_walkscore_ot=max_walkscore_gt50
max_walkscore_ot_geo=max_walkscore_ot.buffer(1760)
max_walkscore_ot.geometry=max_walkscore_ot_geo

In [563]:
Municipalities_File_50to80_ot=geopandas.sjoin(Municipalities_file_50to80,max_walkscore_ot,how='left',op='intersects')
walkscore_ind=list(Municipalities_File_50to80_ot.loc[:,'Municipali']==Municipalities_File_50to80_ot.loc[:,'MUNICIPALITIES FILE MUNICIPALITY'])
ID=Municipalities_File_50to80_ot.loc[:,'ID']
Municipalities_File_50to80_ot=pd.DataFrame({'ID':ID,'walkscore_ind':walkscore_ind})

In [564]:
Municipalities_File_50to80_ot=Municipalities_File_50to80_ot.groupby('ID').agg('sum').reset_index()

In [566]:
Municipalities_File_50to80_ot.loc[:,'walkscore_ind'][Municipalities_File_50to80_ot.loc[:,'walkscore_ind']>0]=7

## Find Parcels 1/2 of a Mile from Max Walkscore

In [570]:
max_walkscore_oh=max_walkscore_gt50
max_walkscore_oh_geo=max_walkscore_oh.buffer(2640)
max_walkscore_oh.geometry=max_walkscore_oh_geo

In [571]:
Municipalities_File_50to80_oh=geopandas.sjoin(Municipalities_file_50to80,max_walkscore_oh,how='left',op='intersects')
walkscore_ind=list(Municipalities_File_50to80_oh.loc[:,'Municipali']==Municipalities_File_50to80_oh.loc[:,'MUNICIPALITIES FILE MUNICIPALITY'])
ID=Municipalities_File_50to80_oh.loc[:,'ID']
Municipalities_File_50to80_oh=pd.DataFrame({'ID':ID,'walkscore_ind':walkscore_ind})

In [572]:
Municipalities_File_50to80_oh=Municipalities_File_50to80_oh.groupby('ID').agg('sum').reset_index()

In [573]:
Municipalities_File_50to80_oh.loc[:,'walkscore_ind'][Municipalities_File_50to80_oh.loc[:,'walkscore_ind']>0]=5

 ## Combine 1/3rd and 1/2 Files
    

In [579]:
gt50=Municipalities_File_50to80_ot.merge(Municipalities_File_50to80_oh,how='left',left_on='ID',right_on='ID')
walkscore_ind=gt50.loc[:,['walkscore_ind_x','walkscore_ind_y']].max(axis=1)
gt50=pd.DataFrame({'ID':gt50.loc[:,'ID'],'walkscore_ind':walkscore_ind})

## Combine All Files

In [584]:
lt50_cities=max_walkscore[max_walkscore['max_walkscore']<50].groupby('MUNICIPALITIES FILE MUNICIPALITY').agg('mean').reset_index()
lt50_all_fields=lt50_cities.merge(Municipalities_file,how='left',left_on='MUNICIPALITIES FILE MUNICIPALITY',\
                                  right_on='Municipali')
walkscore=np.zeros(len(lt50_all_fields))
lt50=pd.DataFrame({'ID':lt50_all_fields.loc[:,'ID'],'walkscore_ind':walkscore})

walkscore=pd.concat([gt80,gt50,lt50])


In [1]:
data_dir = base + r"/data"
fp_output_file = data_dir + r"/walkscore.csv"

walkscore.to_csv(fp_output_file)

# walkscore.to_csv('documents/UofM/SIADS 591/walkscore.csv')

NameError: name 'base' is not defined